# Context dict:

## Patient data
    {
    'Name': str, 'Patient_ID': str, 
    'Age': str(edad)+ 'y', 'Gender': str, 
    'Exam_Date': str(mm-dd-yyyy), 
    'Perf.Physician': str, 
    'Ref.Physician': str, 
    'Operator': str
    }
    
## Measurements (if more than one list of the same type)
    {
    'CARD_card2_2D_General': None, 'IVC_Diam': mm float or list float, 
    'IVSd': mm int, 'LVIDd': mm int , 'LVPWd': mm int, 'IVSs': mm float, 'LVIDs': mm int, 'LVPWs': mm float,
    'Ao_Diam': mm int, 'RAAd': diam cm2 int, 'Distance': mm int, 'LAAd': volume ml/m2 float,
    'A4C_Simpson': % str, 'LVEDV': float ml, 'LVESV': float ml, 'LVAd': float cm2, 'LVLd': float cm2, 'LVAs': float,   'LVLs':float, 
    'CARD_card2_M_General': None, 'TAPSE': float mm, 
    'CARD_card2_Doppler_General': None, 
    'AV_Vmax': float m/seg, 'AV_Vmax_grad': float mmHg, 'PG': float mmHg, 
    'LVOT_Vmax': float m/seg, 'LVOT_Vmax_grad': float mmHg, 
    'MV_Vel_E': float m/seg, 'MV_Vel_E_grad': float mmHg, 'MV_Vel_A': float m/seg, 'MV_Vel_A_grad': float mmHg,
    'Med_Vel_E': float cm/seg, 'Lat_Vel_E': float cm/seg, 'IVRT': float msec,
    'Calculation_Item': 'Result', 'card2': None, 'FS_2D_': % int, 'LVIDs_Index_2D_': ml/m2 float, 'LVd_Mass_2D_ASE_': float gr,
    'LVd_Mass_Index_2D_ASE_': int gr/m2, 'RWT_2D_': dimless %, 'IVS_2D_': 7.16, 'LVPW_2D_': 1.12, 'LVESVI_A4C_Simp_': float ml/m2, 'CSA_AV_SV_': 7.61, 'MV_E_A_Ratio': float, 'AV_VR': float, 'E_Med_E': float, 'E_Lat_E': float, 'Average_E': float, 'E_Avg_E': float
    }
    
## Image

    { 'key': 'imageN',
      'image': docxtpl.inline_image.InlineImage,
      'key': 'imageN+1',
      'image':docxtpl.inline_image.InlineImage,
      ...
    }
    
## MOT

    {
    
    }
 

In [4]:
import nbimporter
from template_manager import template_selector
from patient_data_extraction import extract_patient_info,extract_patient_measurements,image_extractor,mot_extractor
from file_conversion import cargar_path
from aux_calculations import calc_e_e,remove_signs,expand_dict_with_lists_inplace
from docx import Document
from docxtpl import DocxTemplate

def render_template(path)-> None:
    '''
    acepta un path y determina si es cardiaco o stress, si son estos extrae medidas, tratornos de motilidad y mapa polar
    Si no lo es, extrae solamente datos del paciente y las imagenes del archivo docx producido por el equipo vinno
    guarda la renderizaci'on en la misma carpeta donde estaba el docx para poder realizar una comparacion
        
    '''
    doc=Document(path)
    template,tipo=template_selector(path)
    info_pac= extract_patient_info(doc)
    image=image_extractor(doc,template,tipo=tipo)
    #guardo el save path con nombre tipo de estudio y fecha en la misma carpeta.
    save_path=str(path).rsplit('\\',1)[0]+f'\\{info_pac["Name"]} {tipo} {info_pac["Exam Date"]}.docx'
    print(save_path)
    print(tipo)
    #Si es carotida solo van los datos del paciente
    if tipo in ['card','stress']: 
        medidas= extract_patient_measurements(path) 
        if tipo =='stress':
            if isinstance(medidas.get('MV Vel E',None),list):
                expand_dict_with_lists_inplace(medidas)
                medidas['E_e_rel'],medidas['e_e_avg']=calc_e_e(medidas)
            mot=mot_extractor(doc)
            context={**info_pac,**medidas,'image': image['image'], 'mot': mot['mot']}
        
        else:
            context={**info_pac,**medidas,'image': image['image']}
    else: 
        context={**info_pac,'image': image['image']}
        
    #quito los espacios en los context para que no haya conflicto
    context= remove_signs(context)

    #template.render(context)
    #template.save(save_path)
    print(f'informe {info_pac["Name"]} creado con exito')
        

In [2]:
lista_de_trabajo=cargar_path(r"C:\Users\Luciano\Documents\temp\documentos")
lista_de_trabajo

Converted C:\Users\Luciano\Documents\temp\documentos\[aliciacarrera]-[20241021004]-[a4aae7c0-fae9-468f-92e2-dbad0be7918c]\8_F4-12L_Carotid.Report.V3.doc to C:\Users\Luciano\Documents\temp\documentos\[aliciacarrera]-[20241021004]-[a4aae7c0-fae9-468f-92e2-dbad0be7918c]\8_F4-12L_Carotid.Report.V3.docx
Converted C:\Users\Luciano\Documents\temp\documentos\[aliciacarrera]-[20241021004]-[a4aae7c0-fae9-468f-92e2-dbad0be7918c]\9_S1-6P_card2.Report.V3.doc to C:\Users\Luciano\Documents\temp\documentos\[aliciacarrera]-[20241021004]-[a4aae7c0-fae9-468f-92e2-dbad0be7918c]\9_S1-6P_card2.Report.V3.docx
Converted C:\Users\Luciano\Documents\temp\documentos\[beatrizpallero]-[20241021001]-[c0fb0662-d9be-4a59-8348-bc81fec7ee9d]\2_F4-12L_Carotid.Report.V3.doc to C:\Users\Luciano\Documents\temp\documentos\[beatrizpallero]-[20241021001]-[c0fb0662-d9be-4a59-8348-bc81fec7ee9d]\2_F4-12L_Carotid.Report.V3.docx
Converted C:\Users\Luciano\Documents\temp\documentos\[beatrizpallero]-[20241021001]-[c0fb0662-d9be-4a59-

[WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[aliciacarrera]-[20241021004]-[a4aae7c0-fae9-468f-92e2-dbad0be7918c]/8_F4-12L_Carotid.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[aliciacarrera]-[20241021004]-[a4aae7c0-fae9-468f-92e2-dbad0be7918c]/9_S1-6P_card2.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[beatrizpallero]-[20241021001]-[c0fb0662-d9be-4a59-8348-bc81fec7ee9d]/2_F4-12L_Carotid.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[beatrizpallero]-[20241021001]-[c0fb0662-d9be-4a59-8348-bc81fec7ee9d]/3_S1-6P_card2.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[claudiamarcos]-[20241021005]-[fe7d9339-0b4f-48ff-acce-740aba9f717c]/10_F4-12L_Carotid.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Documents/temp/documentos/[claudiamarcos]-[20241021005]-[fe7d9339-0b4f-48ff-acce-740aba9f717c]/11_F4-12L_Carotid.Report.V3.docx'),
 WindowsPath('C:/Users/Luciano/Docum

In [5]:
for i in lista_de_trabajo:
    render_template(i)

C:\Users\Luciano\Documents\temp\documentos\[aliciacarrera]-[20241021004]-[a4aae7c0-fae9-468f-92e2-dbad0be7918c]\carrera graciela alicia carotid 10-21-2024.docx
carotid
{'Name': 'carrera graciela alicia', 'Patient_ID': '20241021004', 'Age': '82y', 'Gender': 'Female', 'Exam_Date': '10-21-2024', 'Perf.Physician': '', 'Ref.Physician': '', 'Operator': '', 'image': [{'key': 'image1', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671EDC70>}, {'key': 'image2', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671EDAC0>}, {'key': 'image3', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671EDB50>}, {'key': 'image4', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153661FBBE0>}, {'key': 'image5', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153661F0D60>}, {'key': 'image6', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671EDA30>}, {'key': 'image7', 'image': <docxtpl.inline_image.InlineImage object at 0x000001

C:\Users\Luciano\Documents\temp\documentos\[claudiamarcos]-[20241021005]-[fe7d9339-0b4f-48ff-acce-740aba9f717c]\marcos elizabeth claudia carotid 10-21-2024.docx
carotid
{'Name': 'marcos elizabeth claudia', 'Patient_ID': '20241021005', 'Age': '65y', 'Gender': 'Female', 'Exam_Date': '10-21-2024', 'Perf.Physician': '', 'Ref.Physician': '', 'Operator': '', 'image': [{'key': 'image1', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671E6910>}, {'key': 'image2', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671E6D00>}, {'key': 'image3', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671E01C0>}, {'key': 'image4', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671E07F0>}, {'key': 'image5', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671E0400>}, {'key': 'image6', 'image': <docxtpl.inline_image.InlineImage object at 0x00000153671E6430>}, {'key': 'image7', 'image': <docxtpl.inline_image.InlineImage object at 0x0000

C:\Users\Luciano\Documents\temp\documentos\[juanperazzo]-[20241021006]-[cbe77e67-ddf3-416a-8f24-ff82f1a8a22b]\perazzo juan card 10-21-2024.docx
card
{'Name': 'perazzo juan', 'Patient_ID': '20241021006', 'Age': '44y', 'Gender': 'Male', 'Exam_Date': '10-21-2024', 'Perf.Physician': '', 'Ref.Physician': '', 'Operator': '', 'CARD_card2_2D_General': None, 'IVC_Diam': 11.13, 'IVSd': 11, 'LVIDd': 42, 'LVPWd': 8, 'IVSs': 12.23, 'LVIDs': 32, 'LVPWs': 12.15, 'Ao_Diam': 37, 'RAAd': 18, 'Bi_plane_LA_A_L': '', 'LAEV': 71.4, 'LAVI': 34, 'A4C_LAA': 20.26, 'A4C_LAL': 56.27, 'A2C_LAA': 20.91, 'A2C_LAL': 50.43, 'A4C_Simpson': '', 'LVEDV': 118.24, 'LVESV': [59.21], 'LVAd': 36.23, 'LVLd': 86.0, 'LVAs': [22.8], 'LVLs': [72.0], 'CARD_card2_Doppler_General': None, 'RVOT_Vmax': 0.44, 'RVOT_Vmax_grad': 0.79, 'PG': 1.98, 'RVSP': '', 'RAP': '', 'TR_Vmax': 1.64, 'TR_Vmax_grad': 10.82, 'AV_Vmax': 1.22, 'AV_Vmax_grad': 5.93, 'LVOT_Vmax': 0.75, 'LVOT_Vmax_grad': 2.26, 'MV_Vel_E': 0.8, 'MV_Vel_E_grad': 2.54, 'MV_Vel_A